### Set up Kafka cluster using docker ###

* https://docs.confluent.io/platform/current/get-started/platform-quickstart.html
* Download YAML file location - https://github.com/confluentinc/cp-all-in-one/tree/7.8.0-post/cp-all-in-one-kraft/docker-compose.yml
* Run "docker compose up -d" command from command prompt
* Go to url http://localhost:9021 and create topic as mentioned in documentation
* Managing multiple python environments - https://medium.com/@adocquin/mastering-python-virtual-environments-with-pyenv-and-pyenv-virtualenv-c4e017c0b173

Another setup Set up using documentation https://hevodata.com/learn/kafka-console-producer/#step1


In [1]:
# Import necesary libraries
import json
import time
import requests
from random import choice
from confluent_kafka import Producer
from confluent_kafka import Consumer

In [7]:
# set config for kafka
config = {
        # User-specific properties that you must set
        'bootstrap.servers': 'localhost:9092',
        # 'sasl.username':     '<CLUSTER API KEY>',
        # 'sasl.password':     '<CLUSTER API SECRET>',

        # Fixed properties
        # 'security.protocol': 'SASL_SSL',
        # 'sasl.mechanisms':   'PLAIN',
        'group.id': 'kafka-python-getting-started',
        'auto.offset.reset': 'earliest'
}

In [ ]:
# Create producer for topic to produce some key value pairs
producer = Producer(config)

# Produce data by selecting random values from these lists.
topic = "purchases"
user_ids = ['eabara', 'jsmith', 'sgarcia', 'jbernard', 'htanaka', 'awalther']
products = ['book', 'alarm clock', 't-shirts', 'gift card', 'batteries']

# Optional per-message delivery callback (triggered by poll() or flush())
# when a message has been successfully delivered or permanently
# failed delivery (after retries).
def delivery_callback(err, msg):
    if err:
        print('ERROR: Message failed delivery: {}'.format(err))
    else:
        print("Produced event to topic {topic}: key = {key:12} value = {value:12}".format(
            topic=msg.topic(), key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))

count = 0
for _ in range(10):
    user_id = choice(user_ids)
    product = choice(products)
    producer.produce(topic, product, user_id, callback=delivery_callback)
    count += 1

# Block until the messages are sent.
producer.poll(10000)
producer.flush()

In [13]:
# Create Consumer instance
consumer = Consumer(config)

# Subscribe to topic
topic = "purchases"
consumer.subscribe([topic])

# Poll for new messages from Kafka and print them.
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            # Initial message consumption may take up to
            # `session.timeout.ms` for the consumer group to
            # rebalance and start consuming
            print("Waiting...")
        elif msg.error():
            print("ERROR: %s".format(msg.error()))
        else:
            # Extract the (optional) key and value, and print.
            print("Consumed event from topic {topic}: key = {key:12} value = {value:12}".format(
                topic=msg.topic(), key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))
except KeyboardInterrupt:
    pass
finally:
    # Leave group and commit final offsets
    consumer.close()

Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...
Waiting...


%6|1735752832.229|FAIL|rdkafka#producer-5| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 1ms in state APIVERSION_QUERY)
%6|1735752832.467|FAIL|rdkafka#producer-5| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY, 1 identical error(s) suppressed)
%3|1735752837.694|FAIL|rdkafka#producer-5| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1735752863.233|FAIL|rdkafka#producer-5| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Connection 